In [ ]:
#This code is published in arcgis server on http://qilin.geog.uw.edu:6080/arcgis/rest/services/erictango/Reclass/GPServer/ReclassEric 
#getting Canopy/Ipython to be able to import each others's packages
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.2\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.2\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.2\\ArcToolbox\\Scripts')
import arcpy

inputFc = arcpy.GetParameterAsText(0) #input feature class
inputTable = arcpy.GetParameterAsText(1) #input reclass table
inputField = arcpy.GetParameterAsText(2) #name field reclassified
outputField = arcpy.GetParameterAsText(3) #name output field
NotFoundValue = arcpy.GetParameterAsText(4) #value used in output field for NULL values

fieldname = arcpy.ValidateFieldName(outputField) #Validate new field name
arcpy.AddField_management(inputFc, fieldname, "DOUBLE") #add the validate field to the output feature class

#update the output field
updateCursor = arcpy.da.UpdateCursor(inputFc, [inputField, fieldname])

for row in updateCursor: #search in reclass table
    searchCursor = arcpy.da.SearchCursor(inputTable, ["lowerbound", "upperbound", "value"])
    row[1] = NotFoundValue #set default Value  NotFoundValue
    updateCursor.updateRow(row)
    for rowTable in searchCursor: #if reclass value exists over write default value
        if row[0] >= rowTable[0] and row[0] <= rowTable[1]:
            row[1] = rowTable[2]
            updateCursor.updateRow(row)
        else:
            pass
    del rowTable
    del searchCursor
del row
del updateCursor